# Chatbot with LangGraph and Groq

This notebook demonstrates how to build a simple chatbot using LangGraph for workflow management and Groq for LLM integration.

## Setup and Dependencies

First, let's install the required packages:

In [ ]:
# Install required packages (run this cell if packages are not installed)
# !pip install langgraph langchain-groq langchain-core python-dotenv

## Import Libraries

In [ ]:
from typing import TypedDict, Annotated
from langgraph import graph
from langgraph.graph import add_messages, StateGraph, END
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

print("✅ All libraries imported successfully!")

## Initialize the LLM

We'll use Groq's Llama3-8b-8192 model for fast inference:

In [ ]:
# Initialize the Groq LLM
llm = ChatGroq(
    temperature=0,  # Low temperature for more consistent responses
    model="llama3-8b-8192"  # Fast and efficient model
)

print(f"✅ LLM initialized with model: {llm.model}")
print(f"Temperature: {llm.temperature}")

## Define the Chat State

We use a TypedDict to define the structure of our chat state:

In [ ]:
class BasicChatState(TypedDict):
    messages: Annotated[list, add_messages]

print("✅ Chat state defined")

## Create the Chatbot Node

This function processes the chat state and generates a response:

In [ ]:
def chatbot(state: BasicChatState):
    """
    Process the chat state and generate a response using the LLM.
    
    Args:
        state: The current chat state containing messages
    
    Returns:
        dict: Updated state with the LLM response
    """
    # Generate response using the LLM
    response = llm.invoke(state["messages"])
    
    return {
        "messages": [response]
    }

print("✅ Chatbot function defined")

## Build the LangGraph

Now we'll create the workflow graph:

In [ ]:
# Create the state graph
graph = StateGraph(BasicChatState)

# Add the chatbot node
graph.add_node("chatbot", chatbot)

# Set the entry point
graph.set_entry_point("chatbot")

# Add edge to end
graph.add_edge("chatbot", END)

# Compile the graph
app = graph.compile()

print("✅ LangGraph workflow created and compiled!")

## Test the Chatbot

Let's test our chatbot with a simple message:

In [ ]:
# Test message
test_message = "Hello! How are you today?"

print(f"User: {test_message}")

# Invoke the chatbot
results = app.invoke({
    "messages": [HumanMessage(content=test_message)]
})

print(f"AI: {results['messages'][-1].content}")

## Interactive Chat Function

Let's create a function for interactive chatting:

In [ ]:
def chat_with_bot():
    """
    Interactive chat function that allows you to have a conversation with the bot.
    Type 'exit' to quit.
    """
    print("🤖 Chatbot is ready! Type 'exit' to quit.")
    print("-" * 50)
    
    while True:
        user_input = input("You: ")
        
        if user_input.lower() == "exit":
            print("👋 Goodbye!")
            break
        
        try:
            results = app.invoke({
                "messages": [HumanMessage(content=user_input)]
            })
            
            print(f"🤖 Bot: {results['messages'][-1].content}")
            print("-" * 50)
            
        except Exception as e:
            print(f"❌ Error: {e}")
            print("-" * 50)

print("✅ Interactive chat function created!")

## Start Interactive Chat

Run the cell below to start chatting with your bot:

In [ ]:
# Uncomment the line below to start interactive chat
# chat_with_bot()

## Summary

This notebook demonstrates:

1. ✅ **LangGraph Setup**: Creating a workflow graph for chat processing
2. ✅ **Groq Integration**: Using Groq's fast LLM models
3. ✅ **State Management**: Managing conversation state with TypedDict
4. ✅ **Interactive Chat**: Building a conversational interface
5. ✅ **Error Handling**: Robust error handling for production use

### Next Steps:

- Set up your `GROQ_API_KEY` in a `.env` file
- Experiment with different models and temperatures
- Add more complex conversation flows
- Integrate with web interfaces or APIs

Happy coding! 🚀